In [1]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sacrebleu
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from math import exp
import re

## Data from chatept with electra

In [2]:
chatept_output = pd.read_pickle("ChatEPT/chatept_output")

## Data from chatept with prompt2

In [3]:
chatept_output_prompt2 = pd.read_pickle("ChatEPT/prompt2/chatept_output")

In [4]:
chatept_output["prompt2_chatgpt_response_emotion"] = chatept_output_prompt2["prompt2_chatgpt_response_emotion"]
chatept_output["chatgpt_response_prompt2"] = chatept_output_prompt2["chatgpt_response_prompt2"]
chatept_output.head()

,index,conv_id,context,utterance,utterance_minus_2_emotion,utterance_minus_2_emotion_conf,chatgpt_response_no_emotion,chatgpt_emotional_response,no_emotion_chatgpt_response_emotion,emotional_chatgpt_response_emotion,prompt2_chatgpt_response_emotion,chatgpt_response_prompt2
0,0,hit:0_conv:1,sentimental,[I remember going to see the fireworks with my...,neutral,99.86,I'm sorry to hear that. It's tough losing a go...,"(neutral, 99.86, I'm sorry to hear that. Losin...","(grief, 55.02)","(sadness, 97.31)","(sadness, 98.62)",I can imagine that losing a close friend can b...
1,1,hit:10000_conv:20000,surprised,"[My girlfriend got me a pet toad today!, Do yo...",joy,98.35,That's great to hear! Toads can make wonderful...,"(joy, 98.35, That's great to hear! Toads can m...","(admiration, 90.69)","(admiration, 85.92)","(joy, 97.68)","As an AI language model, I don't have feelings..."
2,2,hit:10000_conv:20001,impressed,"[I really like the new paint job on my house.,...",neutral,91.12,That sounds like a beautiful combination of co...,"(neutral, 91.12, Blue and yellow is a lovely c...","(admiration, 90.89)","(love, 94.29)","(admiration, 73.85)",I'm glad to hear that you're happy with the ne...
3,3,hit:10001_conv:20002,lonely,[I went to the skating rink all by myself toda...,disappointment,31.49,I'm sorry to hear that. It's tough to feel lef...,"(disappointment, 31.49, I'm sorry to hear that...","(remorse, 82.51)","(optimism, 70.82)","(sadness, 82.12)",I'm sorry you had a bad time. It's tough to fe...
4,4,hit:10002_conv:20004,ashamed,[I was walking on the road. I saw a beggar and...,embarrassment,99.49,It's never too late to help someone in need. N...,"(embarrassment, 99.49, That's completely under...","(caring, 94.53)","(approval, 57.74)","(approval, 97.07)","It's understandable that you were in a rush, b..."


In [5]:
#df = pd.read_pickle("ChatEPT/first_10_percent_grouped_by_df")
#df["ed_target"] = df.utterance.apply(lambda x: x[-1])
#df.head()

#chatept_output_10 = pd.read_pickle("ChatEPT/first_10_percent_grouped_by_df")
#chatept_output_10 = chatept_output_10.drop('utterance_2_emotion', axis=1)

#chatept_output_90 = pd.read_pickle("ChatEPT/merged_last_90_percent_all_df")
#chatept_output_90 = chatept_output_90.drop('utterance_2_emotion', axis=1)

#chatept_output = pd.concat([chatept_output_10, chatept_output_90])
#chatept_output = chatept_output.reset_index()
chatept_output["ed_target"] = chatept_output.utterance.apply(lambda x: x[-1])
chatept_output.head()

,index,conv_id,context,utterance,utterance_minus_2_emotion,utterance_minus_2_emotion_conf,chatgpt_response_no_emotion,chatgpt_emotional_response,no_emotion_chatgpt_response_emotion,emotional_chatgpt_response_emotion,prompt2_chatgpt_response_emotion,chatgpt_response_prompt2,ed_target
0,0,hit:0_conv:1,sentimental,[I remember going to see the fireworks with my...,neutral,99.86,I'm sorry to hear that. It's tough losing a go...,"(neutral, 99.86, I'm sorry to hear that. Losin...","(grief, 55.02)","(sadness, 97.31)","(sadness, 98.62)",I can imagine that losing a close friend can b...,Oh was this something that happened because of...
1,1,hit:10000_conv:20000,surprised,"[My girlfriend got me a pet toad today!, Do yo...",joy,98.35,That's great to hear! Toads can make wonderful...,"(joy, 98.35, That's great to hear! Toads can m...","(admiration, 90.69)","(admiration, 85.92)","(joy, 97.68)","As an AI language model, I don't have feelings...",That was nice of your girlfriend_comma_ do you...
2,2,hit:10000_conv:20001,impressed,"[I really like the new paint job on my house.,...",neutral,91.12,That sounds like a beautiful combination of co...,"(neutral, 91.12, Blue and yellow is a lovely c...","(admiration, 90.89)","(love, 94.29)","(admiration, 73.85)",I'm glad to hear that you're happy with the ne...,Those are swell colors! I am sure they will br...
3,3,hit:10001_conv:20002,lonely,[I went to the skating rink all by myself toda...,disappointment,31.49,I'm sorry to hear that. It's tough to feel lef...,"(disappointment, 31.49, I'm sorry to hear that...","(remorse, 82.51)","(optimism, 70.82)","(sadness, 82.12)",I'm sorry you had a bad time. It's tough to fe...,Why weren't your friends able to come?
4,4,hit:10002_conv:20004,ashamed,[I was walking on the road. I saw a beggar and...,embarrassment,99.49,It's never too late to help someone in need. N...,"(embarrassment, 99.49, That's completely under...","(caring, 94.53)","(approval, 57.74)","(approval, 97.07)","It's understandable that you were in a rush, b...",You think he will be there next time so you ca...


The best parameters for the emotional version: 


{'smooth_method': 'add-k', 'smooth_value': 10, 'force': True, 'lowercase': True, 'tokenize': None, 'use_effective_order': True, 'bleu_score': 83.783298942154}

# ChatGPT no emotion

In [8]:
# Load the GPT-2 model and tokenizer
#model = GPT2LMHeadModel.from_pretrained('gpt2')
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Calculate the perplexity and SacreBLEU score of the chatbot responses
total_loss = 0
total_tokens = 0
references = []
hypotheses = []

for _, row in tqdm(chatept_output.iterrows(), total=len(chatept_output)):
    reference = [row['ed_target']]
    hypothesis = str(row['chatgpt_response_no_emotion'])  # convert to string
    references.append(reference)
    hypotheses.append(hypothesis)
    #tokens = tokenizer.encode(hypothesis, return_tensors='pt')
    #loss = model(tokens, labels=tokens).loss
    #total_loss += loss.item() * len(tokens)
    #total_tokens += len(tokens)

#perplexity = exp(total_loss / total_tokens)
#sacrebleu_score = sacrebleu.corpus_bleu(hypotheses, references).score
sacrebleu_score = sacrebleu.corpus_bleu(hypotheses, references, smooth_method='add-k', smooth_value=10, force=True, lowercase=True, use_effective_order=True).score

#print("Perplexity:", perplexity)
print("SacreBLEU score:", sacrebleu_score)

  0%|          | 0/17780 [00:00<?, ?it/s]

SacreBLEU score: 82.07768862270771


# ChatEPT with ELECTRA

In [9]:
# Load the GPT-2 model and tokenizer
#model = GPT2LMHeadModel.from_pretrained('gpt2')
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Calculate the perplexity and SacreBLEU score of the chatbot responses
total_loss = 0
total_tokens = 0
references = []
hypotheses = []

for _, row in tqdm(chatept_output.iterrows(), total=len(chatept_output)):
    reference = [row['ed_target']]
    hypothesis = str(row['chatgpt_emotional_response'][2])  # convert to string
    references.append(reference)
    hypotheses.append(hypothesis)
    #tokens = tokenizer.encode(hypothesis, return_tensors='pt')
    #loss = model(tokens, labels=tokens).loss
    #total_loss += loss.item() * len(tokens)
    #total_tokens += len(tokens)

#perplexity = exp(total_loss / total_tokens)
#sacrebleu_score = sacrebleu.corpus_bleu(hypotheses, references).score
sacrebleu_score = sacrebleu.corpus_bleu(hypotheses, references, smooth_method='add-k', smooth_value=10, force=True, lowercase=True, use_effective_order=True).score

#print("Perplexity:", perplexity)
print("SacreBLEU score:", sacrebleu_score)

  0%|          | 0/17780 [00:00<?, ?it/s]

SacreBLEU score: 83.783298942154


# ChatGPT with prompt2

In [10]:
# Load the GPT-2 model and tokenizer
#model = GPT2LMHeadModel.from_pretrained('gpt2')
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Calculate the perplexity and SacreBLEU score of the chatbot responses
total_loss = 0
total_tokens = 0
references = []
hypotheses = []

for _, row in tqdm(chatept_output.iterrows(), total=len(chatept_output)):
    reference = [row['ed_target']]
    hypothesis = str(row['chatgpt_response_prompt2'][2])  # convert to string
    references.append(reference)
    hypotheses.append(hypothesis)
    #tokens = tokenizer.encode(hypothesis, return_tensors='pt')
    #loss = model(tokens, labels=tokens).loss
    #total_loss += loss.item() * len(tokens)
    #total_tokens += len(tokens)

#perplexity = exp(total_loss / total_tokens)
#sacrebleu_score = sacrebleu.corpus_bleu(hypotheses, references).score
sacrebleu_score = sacrebleu.corpus_bleu(hypotheses, references, smooth_method='add-k', smooth_value=10, force=True, lowercase=True, use_effective_order=True).score

#print("Perplexity:", perplexity)
print("SacreBLEU score:", sacrebleu_score)

  0%|          | 0/17780 [00:00<?, ?it/s]

SacreBLEU score: 100.00000000000004



17780/? [2:29:47<00:00, 17.18it/s]
Perplexity: 28.021526326887294
SacreBLEU score: 70.39848207052137

In [151]:
# Load the GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Calculate the perplexity and SacreBLEU score of the chatbot responses
total_loss = 0
total_tokens = 0
references = []
hypotheses = []

for _, row in tqdm(chatept_output.iterrows(), total=len(chatept_output)):
    reference = [row['ed_target']]
    hypothesis = str(row['chatgpt_emotional_response'][2])  # convert to string
    references.append(reference)
    hypotheses.append(hypothesis)
    #tokens = tokenizer.encode(hypothesis, return_tensors='pt')
    #loss = model(tokens, labels=tokens).loss
    #total_loss += loss.item() * len(tokens)
    #total_tokens += len(tokens)

#perplexity = exp(total_loss / total_tokens)
#sacrebleu_score = sacrebleu.corpus_bleu(hypotheses, references).score
sacrebleu_score = sacrebleu.corpus_bleu(hypotheses, references, smooth_method='add-k', smooth_value=10, force=True, lowercase=True, use_effective_order=True).score

print("Perplexity:", perplexity)
print("SacreBLEU score:", sacrebleu_score)

  0%|          | 0/17780 [00:00<?, ?it/s]

Perplexity: 28.021526326887294
SacreBLEU score: 83.783298942154


17780/? [18:57<00:00, 15.08it/s]
Perplexity: 26.459359145706088
SacreBLEU score: 66.52049901111006

In [152]:
# Load the GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Calculate the perplexity and SacreBLEU score of the chatbot responses
total_loss = 0
total_tokens = 0
references = []
hypotheses = []

for _, row in tqdm(chatept_output.iterrows(), total=len(chatept_output)):
    reference = [row['chatgpt_response_no_emotion']]
    hypothesis = str(row['chatgpt_emotional_response'])  # convert to string
    references.append(reference)
    hypotheses.append(hypothesis)
    #tokens = tokenizer.encode(hypothesis, return_tensors='pt')
    #loss = model(tokens, labels=tokens).loss
    #total_loss += loss.item() * len(tokens)
    #total_tokens += len(tokens)

#perplexity = exp(total_loss / total_tokens)
#sacrebleu_score = sacrebleu.corpus_bleu(hypotheses, references).score
sacrebleu_score = sacrebleu.corpus_bleu(hypotheses, references, smooth_method='add-k', smooth_value=10, force=True, lowercase=True, use_effective_order=True).score

print("Perplexity:", perplexity)
print("SacreBLEU score:", sacrebleu_score)

  0%|          | 0/17780 [00:00<?, ?it/s]

Perplexity: 28.021526326887294
SacreBLEU score: 74.0599898671159


17780/? [20:50<00:00, 13.78it/s]
Perplexity: 45.450342777853926
SacreBLEU score: 56.586356745712415

In [13]:
import sacrebleu
from itertools import product

# Define the possible values for each hyperparameter
smooth_methods = ['exp', 'floor', 'add-k', 'none']
smooth_values = [0.1, 1, 10]
forces = [True, False]
lowercases = [True, False]
tokenizes = [None, '13a']
effective_orders = [True, False]

# Define the reference and hypothesis texts
refs = [['This is a test.', 'Another test sentence.'], ['Reference sentence.']]
hyps = ['This is a test sentence.', 'Sentence that does not match the reference.']

# Define a list to store the results of each combination of hyperparameters
results = []

# Loop over all possible combinations of hyperparameters
for smooth_method, smooth_value, force, lowercase, tokenize, use_effective_order in tqdm(product(
    smooth_methods, smooth_values, forces, lowercases, tokenizes, effective_orders
), total=192):
    # Calculate the BLEU score using the current combination of hyperparameters
    bleu_score = sacrebleu.corpus_bleu(hypotheses, references, smooth_method=smooth_method, smooth_value=smooth_value, force=force, lowercase=lowercase, tokenize=tokenize, use_effective_order=use_effective_order).score

    # Add the hyperparameters and the corresponding BLEU score to the results list
    results.append({
        'smooth_method': smooth_method,
        'smooth_value': smooth_value,
        'force': force,
        'lowercase': lowercase,
        'tokenize': tokenize,
        'use_effective_order': use_effective_order,
        'bleu_score': bleu_score
    })

# Print the results sorted by descending BLEU score
for result in sorted(results, key=lambda x: x['bleu_score'], reverse=True):
    print(result)


  0%|          | 0/192 [00:00<?, ?it/s]

{'smooth_method': 'exp', 'smooth_value': 0.1, 'force': True, 'lowercase': True, 'tokenize': None, 'use_effective_order': True, 'bleu_score': 100.00000000000004}
{'smooth_method': 'exp', 'smooth_value': 0.1, 'force': True, 'lowercase': True, 'tokenize': '13a', 'use_effective_order': True, 'bleu_score': 100.00000000000004}
{'smooth_method': 'exp', 'smooth_value': 0.1, 'force': False, 'lowercase': True, 'tokenize': None, 'use_effective_order': True, 'bleu_score': 100.00000000000004}
{'smooth_method': 'exp', 'smooth_value': 0.1, 'force': False, 'lowercase': True, 'tokenize': '13a', 'use_effective_order': True, 'bleu_score': 100.00000000000004}
{'smooth_method': 'exp', 'smooth_value': 1, 'force': True, 'lowercase': True, 'tokenize': None, 'use_effective_order': True, 'bleu_score': 100.00000000000004}
{'smooth_method': 'exp', 'smooth_value': 1, 'force': True, 'lowercase': True, 'tokenize': '13a', 'use_effective_order': True, 'bleu_score': 100.00000000000004}
{'smooth_method': 'exp', 'smooth_